# Baseline Model: Implicit Ratings

In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import random

import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler, SequentialSampler 

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Figure out whether we have a GPU; if not use the CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

## Data Import and Processing

In [6]:
feat_df = pd.read_csv('/content/drive/MyDrive/Data_Implict_Ratings/item_feature.csv')

In [7]:
feat_df[feat_df['item_id']==28949]

,item_id,item_feature_id
28949,28949,139


In [112]:
# Load in training data
df_train_full = pd.read_csv("/content/drive/MyDrive/Data_Implict_Ratings/training.csv")

#total_train_2ns_seed0_new
#df_train_full.drop('item_feature_id',axis=1,inplace=True)
df_train_full.drop_duplicates(inplace=True)

##train_df = df_train_full.join(feat_df, on='item_id', how='left', rsuffix='dup').drop('item_iddup', axis=1)
#train_df.head()

In [114]:
df_train_full.drop('context_feature_id',axis=1,inplace=True)

In [17]:
#train_df.drop('item_feature_iddup',axis=1,inplace=True)

In [18]:
#train_df.drop_duplicates(inplace=True)

In [115]:
df_train_full

,user_id,item_id
0,0,28366
1,0,16109
2,0,11500
3,0,20750
4,0,8759
...,...,...
970239,200150,20387
970240,200152,30710
970242,200152,12006
970243,200152,25030


In [122]:
def NegativeSampleUniform(df):
    users=df['user_id'].unique()
    items=df['item_id'].unique()
    final=[]
    for i,user_id in enumerate(list(users)):
      if i%1000==0:
        print(i)
      one_user_data=df[df['user_id'] == user_id] 
      item_already=set(one_user_data['item_id'].unique())
      filtered = set(items).difference(item_already)
      #np.random.seed(3)
      #print(user_id,2*len(one_user_data['item_id']),len((filtered)))
      if user_id==200153:
        neg_samples=random.choices(list(filtered), k=2*len(one_user_data['item_id']))
      else:
        neg_samples=random.sample(list(filtered), 2*len(one_user_data['item_id']))
      user_new=[user_id for k in range(len(neg_samples))]
      #srr=np.array(one_user_data['context_feature_id'])
      #context=np.append(srr,srr)
      check=[(i,j) for i,j in zip(user_new,neg_samples)]
      final.append(check)
    final = [item for sublist in final for item in sublist]
    neg_samples_df=pd.DataFrame(final,columns=['user_id','item_id'])
    #print(neg_samples_df)
    neg_samples_df = neg_samples_df.join(feat_df, on='item_id', how='left', rsuffix='dup').drop('item_iddup', axis=1)
    df = df.join(feat_df, on='item_id', how='left', rsuffix='dup').drop('item_iddup', axis=1)
    df['flag']=1
    neg_samples_df['flag']=0
    df1=pd.concat([df[['user_id','item_id','item_feature_id','flag']],neg_samples_df])
    return df1

In [ ]:
new_df=NegativeSampleUniform(df_train_full)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000


In [24]:
train_df.shape[0]/3

552399.6666666666

In [ ]:
#train_df['item_feature_id']=train_df['item_feature_id'].astype(int)

In [ ]:
#user_freq=pd.DataFrame(train_df['user_id'].value_counts()/sum(train_df['user_id'].value_counts()))
#user_freq

In [ ]:
#items_freq=pd.DataFrame(train_df['item_id'].value_counts()/sum(train_df['item_id'].value_counts()))


In [ ]:
#train_df = train_df.join(user_freq, on='user_id', how='left', rsuffix='dup').rename(columns={'user_iddup':'user_freq'})
##train_df = train_df.join(items_freq, on='item_id', how='left', rsuffix='dup').rename(columns={'item_iddup':'items_freq'})
#train_df.head()


In [25]:
# Load in test data
df_test = pd.read_csv('/content/drive/MyDrive/Data_Implict_Ratings/test_kaggle.csv')
df_test = df_test.join(feat_df, on='item_id', how='left', rsuffix='dup').drop('item_iddup', axis=1)
df_test.head(5)

,id,user_id,item_id,context_feature_id,item_feature_id
0,0,4,16835,2,142
1,1,4,22590,3,142
2,2,4,1978,1,142
3,3,4,28916,1,148
4,4,4,14427,2,63


In [ ]:
##test_user_freq=pd.DataFrame(df_test['user_id'].value_counts()/sum(df_test['user_id'].value_counts()))
#test_items_freq=pd.DataFrame(df_test['item_id'].value_counts()/sum(df_test['item_id'].value_counts()))

In [ ]:
#df_test = df_test.join(test_user_freq, on='user_id', how='left', rsuffix='dup').rename(columns={'user_iddup':'user_freq'})
#df_test = df_test.join(test_items_freq, on='item_id', how='left', rsuffix='dup').rename(columns={'item_iddup':'items_freq'})
#df_test.tail()

In [26]:
df_test.tail()

,id,user_id,item_id,context_feature_id,item_feature_id
381380,381380,200151,1702,1,139
381381,381381,200151,21632,1,130
381382,381382,200151,30477,1,130
381383,381383,200151,30477,1,130
381384,381384,200151,17715,1,138


In [27]:
train_df = train_df.sample(frac=1).reset_index(drop=True)

In [28]:
# Sample a third of the data
unknowns = train_df.sample(frac = 0.1,random_state=3,replace=False)#.drop('level_0',axis=1)
# Change the user ids to unknown value
unknowns['user_id'] = 200153

#train_df = pd.concat([train_df, unknowns]).reset_index()
train_df=train_df.append(unknowns).reset_index()

unknowns_items = train_df.sample(frac = 0.01,random_state=3,replace=False)#.drop('level_0',axis=1)
unknowns_items['item_id'] = 39901

train_df=train_df.append(unknowns_items).reset_index()
train_df.drop('level_0',axis=1,inplace=True)

train = train_df.sample(frac = 0.8,random_state=3,replace=False)
val = train_df.drop(train.index)

# Change values in val and test to unknown if missing
df_test.loc[~df_test['user_id'].isin(train_df['user_id'].unique()), 'user_id'] = 200153
df_test.loc[~df_test['item_id'].isin(train_df['item_id'].unique()), 'item_id'] = 39901
#val.loc[~val['user_id'].isin(train['user_id'].unique()), 'user_id'] = 200153

# Stack generated unknowns onto the training


In [ ]:
#train.to_csv('/content/drive/MyDrive/Data_Implict_Ratings/train.csv', index=False)
#val.to_csv('/content/drive/MyDrive/Data_Implict_Ratings/val.csv', index=False)

In [29]:
df_train_full.shape

(1722520, 5)

In [30]:
len(train)+len(val),len(train_df),(len(df_train_full)+0.1001*len(df_train_full))

(1841148, 1841148, 1894944.2519999999)

In [31]:
#sum(val['flag'])/len(val['flag']),sum(train['flag'])/len(train['flag'])

In [32]:
len(train)+len(val),len(train_df),len(unknowns)

(1841148, 1841148, 165720)

In [33]:
train['user_id'].unique()

array([  5706,  26281, 140223, ...,  94709, 176403,  43683])

In [34]:
len(train[train['user_id']==200153]['user_id'].unique())

1

In [35]:
len(train[train['user_id']==200153]),len(train[train['item_id']==39901])

(133935, 14648)

In [36]:
len(train[train['user_id']==200153])/len(train),len(val[val['user_id']==200153])/len(val),len(df_test[df_test['user_id']==200153])/len(df_test)

(0.09093174229658406, 0.09081009151888765, 0.82192797304561)

In [37]:
len(train[train['item_id']==39901])/len(train),len(val[val['item_id']==39901])/len(val),len(df_test[df_test['item_id']==39901])/len(df_test)

(0.009944884915521434, 0.009724900198245661, 0.0719010973163601)

In [40]:
num_users = max(train_df['user_id'])+1
num_items = max(train_df['item_id'])+1
#num_context = max(train_df['context_feature_id'])+1
num_feats = int(max(feat_df['item_feature_id']))+1

In [41]:
num_users,num_items,num_feats

(200154, 39902, 195)

In [ ]:
# MF with bias model
class MF_bias(nn.Module):
    def __init__(self, num_users, num_items, emb_size=100):
        super(MF_bias, self).__init__()
        self.user_emb = nn.Embedding(num_users, emb_size) # .to(device)
        self.user_bias = nn.Embedding(num_users, 1) # .to(device)
        self.item_emb = nn.Embedding(num_items, emb_size) # .to(device)
        self.item_bias = nn.Embedding(num_items, 1) # .to(device)
        # init 
        self.user_emb.weight.data.uniform_(0,0.05)
        self.item_emb.weight.data.uniform_(0,0.05)
        self.user_bias.weight.data.uniform_(-0.01,0.01)
        self.item_bias.weight.data.uniform_(-0.01,0.01)
        
    def forward(self, u, v):
        # u = u.to(device)
        # v = v.to(device)
        U = self.user_emb(u)
        V = self.item_emb(v)
        b_u = self.user_bias(u).squeeze()
        b_v = self.item_bias(v).squeeze()
        return (U*V).sum(1) +  b_u  + b_v

def valid_metrics_bias(model, valid_dl):
    """Computes validation loss and accuracy"""
    ### BEGIN SOLUTION
    users, items, feats,contexts,user_freq,items_freq, y = next(iter(valid_dl))
    y = y.unsqueeze(1)
    y_hat = model(users, items)
    valid_loss = F.binary_cross_entropy_with_logits(y_hat, y.float())

    y_hat_new = torch.where(y_hat>0, 1, 0)
    valid_acc_int = torch.sum(y_hat_new == y)
    valid_acc = (valid_acc_int*1.00)/len(y)
    ### END SOLUTION
    return float(valid_loss.cpu().detach().numpy()), valid_acc

# here we are not using data loaders because our data fits well in memory
def train_epocs_bias(model, train_dl, valid_dl, epochs=10, lr=0.01, wd=0.0):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
    for i in range(epochs):
        model.train()
        users, items, feats,contexts,user_freq,items_freq, y = next(iter(train_dl))
        y = y.unsqueeze(1)
        y_hat = model(users, items)
        loss = F.binary_cross_entropy_with_logits(y_hat, y.float())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        model.eval()
        train_loss = loss.item()
        valid_loss, valid_acc = valid_metrics_bias(model, valid_dl)
        print("train loss %.3f valid loss %.3f valid acc %.3f" % (train_loss, valid_loss, valid_acc))

## NN Full Model

In [86]:
class MF_feat(nn.Module):
    def __init__(self,num_users,num_items,num_feats, hid_layer=50, emb_size=100, p=0.01):  #user_embeds,item_embeds
        super(MF_feat, self).__init__()
        # Create embeddings
        self.user_emb = nn.Embedding(num_users, emb_size)
        self.item_emb = nn.Embedding(num_items, emb_size)
        self.feats_emb = nn.Embedding(num_feats, emb_size)
       # self.contexts_emb = nn.Embedding(num_contexts, emb_size)
        # Initlializing weights
       # self.user_emb.weight.data=user_embeds
        #self.item_emb.weight.data=item_embeds
        self.user_emb.weight.data.uniform_(0,0.05)
        self.item_emb.weight.data.uniform_(0,0.05)
        self.feats_emb.weight.data.uniform_(0,0.05)
        #self.contexts_emb.weight.data.uniform_(0,0.05)
        # Initialize layers
        self.linear1 = nn.Linear(3*emb_size, hid_layer,bias =True)
        self.linear2 = nn.Linear(hid_layer, int(hid_layer/2),bias =True)
        self.linear3 = nn.Linear(int(hid_layer/2)+emb_size, 1,bias =True) #
        self.dropout = nn.Dropout(p)
        
    def forward(self, u, i, f):
        u = self.user_emb(u)
        i = self.item_emb(i)
        f = self.feats_emb(f)
        #c = self.contexts_emb(c)
        gmf = u * i
        x = torch.cat((u,i,f),1)
        x = self.linear1(x)
        x = self.dropout(x)
        x = F.relu(x)
        #x = self.linear2(x)
        #x = self.dropout(x)
        #x = F.relu(x)
        #ufq=torch.unsqueeze(ufq, 1)
        #ifq=torch.unsqueeze(ifq, 1)
       # x= torch.cat((x,f,c),1)
        x = self.linear2(x)
        x = self.dropout(x)
        x = F.relu(x)
        z= torch.cat((gmf,x),1)
        y = self.linear3(z)
        return y


class UserDataset(Dataset):
    def __init__(self, df):
        u = df['user_id'].values
        i = df['item_id'].values
        f = df['item_feature_id'].values
        #c = df['context_feature_id'].values
        y = df['flag'].values

        self.u = torch.LongTensor(u)
        self.i = torch.LongTensor(i)
        self.f = torch.LongTensor(f)
        #self.c = torch.LongTensor(c)
        self.y = torch.LongTensor(y)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, index):
        return self.u[index], self.i[index], self.f[index],self.y[index] #, self.c[index]


def train_epocs(model2,train_dl, valid_dl, epochs=10, lr=0.01, wd=0.0):
    optimizer = torch.optim.Adam(model2.parameters(), lr=lr, weight_decay=wd)
    #val=valid_dl.sample(frac = 1,replace=False)
    #val=NegativeSampleUniform(val,items_total)
    #users, items, feats,contexts, y =convert_data(val)
    for i in range(epochs):
        model2.train()
        #train_new=train_dl.sample(frac = 0.01,replace=False)
        #print(len(train))
        #print(train_new)
        #train=NegativeSampleUniform(train_new,items_total)
        #print(UserDataset(train))
        users, items, feats, y = next(iter(train_dl))#. #=convert_data(train)
        y = y.unsqueeze(1)
        y_hat = model2(users, items, feats)
        loss = F.binary_cross_entropy_with_logits(y_hat, y.float())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        model2.eval()
        train_loss = loss.item()
        valid_loss, valid_acc = valid_metrics(model2,valid_dl)
        print("train loss %.3f valid loss %.3f valid acc %.3f" % (train_loss, valid_loss, valid_acc))

def valid_metrics(model2,valid_dl):
    """Computes validation loss and accuracy"""
    ### BEGIN SOLUTION
     #user_freq,items_freq
    users, items, feats, y = next(iter(valid_dl))
    y = y.unsqueeze(1)
    y_hat = model2(users, items, feats)
    valid_loss = F.binary_cross_entropy_with_logits(y_hat, y.float())

    y_hat_new = torch.where(y_hat>0, 1, 0)
    valid_acc_int = torch.sum(y_hat_new == y)
    valid_acc = (valid_acc_int*1.00)/len(y)
    ### END SOLUTION
    return float(valid_loss.cpu().detach().numpy()), valid_acc

In [87]:
#train=NegativeSample(train)
#val=NegativeSample(val)

In [103]:
# # Create Datasets
train_dl = DataLoader(UserDataset(train), batch_size=int(len(train)/10), shuffle=True)
valid_dl = DataLoader(UserDataset(val), batch_size=len(val), shuffle=False)#. #int(len(val))

In [104]:
#users, items, feats,contexts= next(iter(train_dl))

In [105]:
model2 = MF_feat(num_users,num_items,num_feats,hid_layer=50,emb_size=100,p=0.3) # .to(device)

In [106]:
#num_users,num_items,num_feats,num_context

In [107]:
train_epocs(model2,train_dl, valid_dl, epochs=10, lr=0.1, wd=1e-5)

train loss 0.693 valid loss 0.643 valid acc 0.522
train loss 0.668 valid loss 0.326 valid acc 1.000
train loss 0.334 valid loss 0.227 valid acc 1.000
train loss 0.225 valid loss 0.009 valid acc 1.000
train loss 0.031 valid loss 0.001 valid acc 1.000
train loss 0.005 valid loss 0.002 valid acc 1.000
train loss 0.004 valid loss 0.004 valid acc 1.000
train loss 0.010 valid loss 0.006 valid acc 1.000
train loss 0.010 valid loss 0.008 valid acc 1.000
train loss 0.010 valid loss 0.012 valid acc 1.000


In [ ]:
train_epocs(model2, train_dl, valid_dl, epochs=5, lr=0.1, wd=1e-5)

In [ ]:
train_epocs(model2, train_dl, valid_dl, epochs=5, lr=0.01, wd=1e-5)

In [ ]:
train_epocs(model2, train_dl, valid_dl, epochs=5, lr=0.01, wd=1e-5)

In [ ]:
train_epocs(model2, train_dl, valid_dl, epochs=5, lr=0.01, wd=1e-5)

train loss 0.437 valid loss 0.403 valid acc 0.862
train loss 0.415 valid loss 0.391 valid acc 0.866
train loss 0.405 valid loss 0.381 valid acc 0.868
train loss 0.398 valid loss 0.371 valid acc 0.869
train loss 0.392 valid loss 0.364 valid acc 0.870


In [ ]:
train_epocs(model2, train_dl, valid_dl, epochs=5, lr=0.001)

train loss 0.329 valid loss 0.317 valid acc 0.875
train loss 0.327 valid loss 0.317 valid acc 0.875
train loss 0.327 valid loss 0.317 valid acc 0.875
train loss 0.327 valid loss 0.317 valid acc 0.875
train loss 0.326 valid loss 0.317 valid acc 0.875


In [ ]:
train_epocs(model2, train_dl, valid_dl, epochs=5, lr=0.0001)

train loss 0.323 valid loss 0.317 valid acc 0.875
train loss 0.326 valid loss 0.317 valid acc 0.875
train loss 0.324 valid loss 0.317 valid acc 0.875
train loss 0.325 valid loss 0.317 valid acc 0.875
train loss 0.324 valid loss 0.317 valid acc 0.875


## Make Predictions

In [110]:
# Create a data loader
users = torch.LongTensor(df_test['user_id'].values)
items = torch.LongTensor(df_test['item_id'].values)
context = torch.LongTensor(df_test['context_feature_id'].values)
feats = torch.LongTensor(df_test['item_feature_id'].values)
#user_freqs = torch.LongTensor(df_test['user_freq'].values)
#item_freqs = torch.LongTensor(df_test['items_freq'].values)

In [111]:
# Make predictions
flags = model2(users, items, feats)
df_test['rating'] = torch.sigmoid(flags).squeeze().detach().numpy()
submission = df_test[['id','user_id','rating']]
submission.describe()

,id,user_id,rating
count,381385.000000,381385.000000,381385.000000
mean,190692.000000,182318.747077,0.997100
std,110096.510545,45407.208780,0.053629
min,0.000000,10.000000,0.000000
25%,95346.000000,200153.000000,1.000000
50%,190692.000000,200153.000000,1.000000
75%,286038.000000,200153.000000,1.000000
max,381384.000000,200153.000000,1.000000


In [ ]:
submission.loc[submission['user_id']!=200153.000000, 'rating'].mean(),submission.loc[submission['user_id']==200153.000000, 'rating'].mean()

(0.47126829624176025, 0.5506588220596313)

In [ ]:
submission.drop(['user_id'],axis=1,inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
submission.to_csv('/content/drive/MyDrive/Data_Implict_Ratings/Mar3_new_try1.csv', index=False)